### This file generate Hafez poems

In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import glob

### Read the data



In [4]:
path_to_file = 'hafez-d.txt'

In [6]:

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print (f'Length {len(text)}')

execluded = '!()*-.1:=[]«»;؛,،~?؟#\u200f\ufeff'
out = ""

for char in text:
  if char not in execluded:
    out += char

text = out
text = text.replace("\t\t\t", "\t")
text = text.replace("\r\r\n", "\n")
text = text.replace("\r\n","\n")
text = text.replace("\t\n", "\n")
text = text.replace("\n\n", "\n")
text = '\n'.join(line for line in text.split('\n') if len(line) >= 7)


vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

print(vocab)

Length 289560
38 unique characters
['\n', ' ', 'Y', 'آ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c']


In [10]:
print(text[:349])

ساقی به نور باده برافروز جام ما
مطرب بگو که کار جهان شد به کام ما
ما در پياله عکس رخ يار ديده‌ايم
ای بی‌خبر ز لذت شرب مدام ما
هرگز نميرد آن که دلش زنده شد به عشق
ثبت است بر جريده عالم دوام ما
چندان بود کرشمه و ناز سهی قدان
کايد به جلوه سرو صنوبرخرام ما
ای باد اگر به گلشن احباب بگذری
زنهار عرضه ده بر جانان پيام ما
گو نام ما ز ياد به عمدا چه می‌بری



### Vectorize the text



In [11]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print(text_as_int[:10])

[15  4 24 36  1  5 28  1 27 29]


In [14]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:5s}  :  {:5d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n'   :      0,
  ' '    :      1,
  'Y'    :      2,
  'آ'    :      3,
  'ا'    :      4,
  'ب'    :      5,
  'ت'    :      6,
  'ث'    :      7,
  'ج'    :      8,
  'ح'    :      9,
  'خ'    :     10,
  'د'    :     11,
  'ذ'    :     12,
  'ر'    :     13,
  'ز'    :     14,
  'س'    :     15,
  'ش'    :     16,
  'ص'    :     17,
  'ض'    :     18,
  'ط'    :     19,
  ...
}


In [17]:
print ('{} - {}'.format(repr(text[:11]), text_as_int[:11]))

'ساقی به نور' - [15  4 24 36  1  5 28  1 27 29 13]


In [18]:
seq_length = 200
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

س
ا
ق
ی
 


In [19]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'ساقی به نور باده برافروز جام ما\nمطرب بگو که کار جهان شد به کام ما\nما در پياله عکس رخ يار ديده\u200cايم\nای بی\u200cخبر ز لذت شرب مدام ما\nهرگز نميرد آن که دلش زنده شد به عشق\nثبت است بر جريده عالم دوام ما\nچندان بود'
' کرشمه و ناز سهی قدان\nکايد به جلوه سرو صنوبرخرام ما\nای باد اگر به گلشن احباب بگذری\nزنهار عرضه ده بر جانان پيام ما\nگو نام ما ز ياد به عمدا چه می\u200cبری\nخود آيد آن که ياد نياری ز نام ما\nمستی به چشم شاهد دلب'
'ند ما خوش است\nزان رو سپرده\u200cاند به مستی زمام ما\nترسم که صرفه\u200cای نبرد روز بازخواست\nنان حلال شيخ ز آب حرام ما\nحافظ ز ديده دانه اشکی همی\u200cفشان\nباشد که مرغ وصل کند قصد دام ما\nدريای اخضر فلک و کشتی هلال\nهستند'
' غرق نعمت حاجی قوام ما\nای فروغ ماه حسن از روی رخشان شما\nآب روی خوبی از چاه زنخدان شما\nعزم ديدار تو دارد جان بر لب آمده\nبازگردد يا برآيد چيست فرمان شما\nکس به دور نرگست طرفی نبست از عافيت\nبه که نفروشند م'
'ستوری به مستان شما\nبخت خواب آلود ما بيدار خواهد شد مگر\nزان که زد بر ديده آبی روی رخشان شما\nبا صبا همراه بفرست از رخت گلدسته\u

In [20]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ساقی به نور باده برافروز جام ما\nمطرب بگو که کار جهان شد به کام ما\nما در پياله عکس رخ يار ديده\u200cايم\nای بی\u200cخبر ز لذت شرب مدام ما\nهرگز نميرد آن که دلش زنده شد به عشق\nثبت است بر جريده عالم دوام ما\nچندان بو'
Target data: 'اقی به نور باده برافروز جام ما\nمطرب بگو که کار جهان شد به کام ما\nما در پياله عکس رخ يار ديده\u200cايم\nای بی\u200cخبر ز لذت شرب مدام ما\nهرگز نميرد آن که دلش زنده شد به عشق\nثبت است بر جريده عالم دوام ما\nچندان بود'


### Create training batches



In [24]:
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 200), dtype=tf.int64, name=None), TensorSpec(shape=(128, 200), dtype=tf.int64, name=None))>

## Build The Model

In [25]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [26]:
def build_model(vocab_size_, embedding_dim_, rnn_units_, batch_size):
  model_ = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_, embedding_dim_,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units_,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size_)
  ])
  return model_

In [27]:
model = build_model(
  vocab_size_ = len(vocab),
  embedding_dim_=embedding_dim,
  rnn_units_=rnn_units,
  batch_size=BATCH_SIZE)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 256)          9728      
                                                                 
 gru (GRU)                   (128, None, 1024)         3938304   
                                                                 
 dense (Dense)               (128, None, 38)           38950     
                                                                 
Total params: 3,986,982
Trainable params: 3,986,982
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [34]:
model.compile(
    optimizer='adam',
    loss = loss,
    metrics=['accuracy'])

### Configure checkpoints

In [35]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [36]:
EPOCHS=400

In [ ]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
                    callbacks=[checkpoint_callback])

Epoch 1/400
10/10 [==============================] - 5s 214ms/step - loss: 4.5535 - accuracy: 0.1803
Epoch 2/400
10/10 [==============================] - 2s 196ms/step - loss: 3.4888 - accuracy: 0.1695
Epoch 3/400
10/10 [==============================] - 2s 196ms/step - loss: 3.2678 - accuracy: 0.2250
Epoch 4/400
10/10 [==============================] - 2s 194ms/step - loss: 2.8923 - accuracy: 0.2307
Epoch 5/400
10/10 [==============================] - 2s 197ms/step - loss: 2.7583 - accuracy: 0.2585
Epoch 6/400
10/10 [==============================] - 2s 197ms/step - loss: 2.6607 - accuracy: 0.2715
Epoch 7/400
10/10 [==============================] - 2s 198ms/step - loss: 2.5939 - accuracy: 0.2889
Epoch 8/400
10/10 [==============================] - 2s 196ms/step - loss: 2.5448 - accuracy: 0.2940
Epoch 9/400
10/10 [==============================] - 2s 199ms/step - loss: 2.5092 - accuracy: 0.2944
Epoch 10/400
10/10 [==============================] - 2s 200ms/step - loss: 2.4825 - accura

## Generate text

In [37]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [1]:
def generate_text(model_, start_string):

  num_generate = 300

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model_.reset_states()
  for i in range(num_generate):
      predictions = model_(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return start_string + ''.join(text_generated)

In [ ]:
generate_poetry = generate_text(model, start_string=u"گفتم ای سلطان خوبان رحم کن بر اين غريب")
print(generate_poetry)

گفتم ای سلطان خوبان رحم کن بر اين غريب
گفت در حلقه آمد به جهان نقش خرابی دارد
در خرابات بگوييد که هشيار کجاست
آن کس است اهل بشارت که اشارت داند
نکته‌ها هست بسی محرم اسرار کجاست
هر سر موی مرا با تو هزاران کار است
ما کجاييم و ملامت گر بی‌کار کجاست
بازآی که بی روی تو ای شمع دل افروز
در بزم حريفان اثر نور و صفا نيست
تيمان غانيت ای گدايی همت


In [ ]:
model.save('hafez.h5')